In [1]:
import os

In [2]:
# Get the list of all files and directories


In [49]:
import tkinter as tk
from tkinter import *
from tkinter import ttk

class TreeViewList(tk.Frame):

    def __init__(self, master,width,height,dataTable,idxs=[]):
        self.master = master
        self.dataTable = dataTable
        tk.Frame.__init__(self, self.master,width=width,height=height,) # , bg="red")
        self.terminal_tree = ttk.Treeview(self)
        self.pack(fill='both', expand=True)
        self.columns = ('vocab_size', 'window_size', 'embedding_dim','hidden_dim','dense_dim',
            'n_layers','max_norm','n_samples','n_epochs','batch_size','train_loss','val_loss')
        self.update_idletasks()
        self.treeColumnWidth = int(self.winfo_width()/len(self.columns))
        self.create_widgets()
        self.fillData()
        
        
    def getTree(self):
        return self.terminal_tree
    
    def fillData(self):
        for record in self.dataTable:
            self.terminal_tree.insert('',tk.END,values=record)
    
    def treeview_sort_column(self,tv, col, reverse):
        l = [(tv.set(k, col), k) for k in tv.get_children('')]
        l.sort(reverse=reverse)

        # rearrange items in sorted positions
        for index, (val, k) in enumerate(l):
            tv.move(k, '', index)

        # reverse sort next time
        tv.heading(col, command=lambda: \
            self.treeview_sort_column(tv, col, not reverse))
    
    def create_widgets(self):
        # button to request data
        # self.request_button = tk.Button(self, text="Send") #, command=self.req_client)
        # self.request_button.grid(row=0, column=0, rowspan=2)

        # timer label
        # self.timer_label = tk.Label(self, text="Timer Settings")
        # self.timer_label.grid(row=0, column=1, pady=3, sticky=tk.NSEW)

        # create tk variable
        # self.timervar = tk.StringVar(self)

        # dropdown dictionary
        # self.timerDict = {"-", "5 minutes", "10 minutes", "15 minutes"}
        # self.timervar.set("-")  # <-- set the default value

        # timer dropdown menu
        # self.timer_option = tk.OptionMenu(self, self.timervar, *self.timerDict) #, command=self.req_timer)
        # self.timer_option.grid(row=1, column=1, padx=3, pady=3, sticky=tk.NSEW)

        # scroll bar for the terminal outputs
        self.terminal_scrollbar = tk.Scrollbar(self)
        self.terminal_scrollbar.grid(row=2, column=5, sticky=tk.NS)

        # terminal output
        
        self.terminal_tree.grid(row=2, column=0, columnspan=3, sticky=tk.NSEW)
        self.terminal_tree.configure(yscrollcommand=self.terminal_scrollbar.set)
        self.terminal_tree["columns"] = self.columns
        self.terminal_tree['show'] = 'headings'
        # self.terminal_tree.column("1", width=100, anchor='c')
        # self.terminal_tree.heading("1", text="Example")
        for col in self.columns:
            words =  col.split('_')
            txt =''
            for w in words:
                txt+=w[0].capitalize()
            self.terminal_tree.heading(col, text=txt, command=lambda:\
                self.treeview_sort_column(self.terminal_tree, col, False))
            self.terminal_tree.column(col,minwidth=30,width=self.treeColumnWidth,stretch=YES,anchor=CENTER)
        
        self.columnconfigure(2, weight=1) # column with treeview
        self.rowconfigure(2, weight=1) # row with treeview        



In [50]:
class Main():
    def __init__(self,parent):
        self.mainTable = []
        self.readFiles()
        self.selectedIndex = -1
        self.topFrame = Frame(root, width=1350, height=700) 
        self.topFrame.pack(side=TOP, fill=X, expand=1, anchor=N)
        titleLabel = Label(self.topFrame, font=('calibri', 12, 'bold'),
                        text="Main List of Runs",
                        bd=5, anchor=W)
        titleLabel.pack(side=TOP)
        self.Bottom = Frame(root, width=1350, height=700, bd=4, relief="ridge")
        self.Bottom.pack(side=BOTTOM, fill=X, expand=1, anchor=S)
        titleLabel = Label(self.Bottom, font=('calibri', 12, 'bold'),
                        text="Result List of Runs",
                        bd=5, anchor=W)
        titleLabel.pack(side=TOP)
        treeListView = TreeViewList(self.topFrame,width=1350,height=350,dataTable=self.mainTable)
        self.tree = treeListView.getTree()
        self.tree.bind("<Double-1>", self.onDoubleClick)
    

    def oneDifFilter(self,index,table):
        list = []
        idxs = []
        for idx in range(len(table)):
            if idx == index:
                continue
            else:
                counter = 0
                for jdx in range(len(table[0])):
                    if table[idx][jdx] != table[index][jdx]:
                        counter+=1
                if counter == 3:
                    list.append(table[idx])
                    idxs.append(idx)

        return list,idxs

    def onDoubleClick(self,event):
        self.Bottom.destroy()
        item = self.tree.selection()[0]
        newTable,idxs = self.oneDifFilter(self.tree.index(item),self.mainTable)
        self.Bottom = Frame(root, width=1350, height=700, bd=4, relief="ridge")
        self.Bottom.pack(side=BOTTOM, fill=X, expand=1, anchor=S)
        titleLabel = Label(self.Bottom, font=('calibri', 12, 'bold'),
                        text="Result List of Runs",
                        bd=5, anchor=W)
        titleLabel.pack(side=TOP)
        TreeViewList(self.Bottom,1350,700,newTable,idxs=idxs)

    def readFiles(self):
        path = "./embedding/"
        dir_list = os.listdir(path)
        for record in dir_list:
            self.mainTable.append(record.replace('.csv','').split('_'))
        

In [53]:
root = Tk()
root.state("normal")
root.title("NLPF Analyzer")
root.configure(bg="grey20")
Main(root)
root.mainloop()